In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import pandas as pd
import numpy as np
import time

In [5]:
    
opts = Options()
opts.add_argument("Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36")
#opts.add_argument('headless')

driver = webdriver.Chrome(options=opts)
time.sleep(5)

#start driver on main url, 
#url_main = 'https://www.glassdoor.com/Explore/browse-companies.htm?overall_rating_low=0&page=1&isHiringSurge=0' #main url
url_main = 'https://www.glassdoor.com/Explore/browse-companies.htm?overall_rating_low=0&page=1&locId=3&locType=N&locName=Canada&filterType=RATING_OVERALL'

driver.get(url_main)
#time.sleep(10)

In [8]:

unsuccessful_links = [] ##UPDATE## this line to create unique list for this scrape attempt
companies = [] ##UPDATE## this line to create unique list for this scrape attempt


def scraping_pages(num_pages):
    #Creating 'n' urls with url_roots to scrape
    url_root = 'https://www.glassdoor.com/Explore/browse-companies.htm?overall_rating_low=0&page=1&locId=3&locType=N&locName=Canada&filterType=RATING_OVERALL' #root url
    nums = [x+1 for x in range(num_pages)] ##UPDATE## x + __ according to where last scrape attempt left off
    url_mains = list(map(lambda n: url_root + str(n), nums)) #adding 'n' number to call url_root 
    time.sleep(10) #give page plenty of time to load (page 1 loads first, then specified 'n' page)
    
    for u in url_mains:
        driver.get(u)
        time.sleep(10)
        
    #looking for 'Overview' links from each main search page
        elems = driver.find_elements_by_tag_name('a') #find links on an individual search page tagged with the 'a' tag
        company_links = []
        for elem in elems:
            company_link = elem.get_attribute('href') #returns every item with 'href' attribute (these are the links for each company)
            if 'Overview' in company_link:
                company_links.append(company_link) #each company's 'Overview' link added to company_link list  
    
    #iterating through each company's "Overview" url
        for url in company_links:
            try: #fail safe for inevitable errors
                driver.get(url)
                time.sleep(5)

                name = 'blackiemaj@gmail.com' # <---- ENTER GLASSDOOR CREDENTIALS HERE
                pw = 'Monday123@'
                
                try: #login                    
                    username = driver.find_element_by_id("userEmail")
                    password = driver.find_element_by_id("userPassword")
                    submit = driver.find_element_by_xpath('//*[@id="InlineLoginModule"]/div/div[2]/div/div[1]/div[3]/form/div[3]/div[1]/button')
                    username.send_keys(name)
                    password.send_keys(pw)
                    submit.click()
                    time.sleep(4) #let page load
                except: #no login required
                    time.sleep(2)
                    pass

##---------------------------------- Gathering Variables - Main Page ---------------------------------##                                
                name = driver.find_element_by_xpath('//*[@id="EmpHeroAndEmpInfo"]/div[3]/div[2]').text
                size = driver.find_element_by_xpath('//*[@id="EIOverviewContainer"]/div/div[1]/ul/li[3]/div').text
                headquarters = driver.find_element_by_xpath('//*[@id="EIOverviewContainer"]/div/div[1]/ul/li[2]/div').text
                industry = driver.find_element_by_xpath('//*[@id="EIOverviewContainer"]/div/div[1]/ul/li[6]/div').text
                try:
                    num_reviews = driver.find_element_by_xpath('//*[@id="EIOverviewContainer"]/div/div[3]/div[3]/a').text
                except: 
                    num_reviews = driver.find_element_by_xpath('//*[@id="EIOverviewContainer"]/div/div[4]/div[3]/a').text        

            #Gather Description - handling "Read More" button
                try:
                    read_more = driver.find_element_by_class_name('css-1tgo67c.e16x8fv00') #button class 
                    read_more.click()
                    time.sleep(2)
                    description = driver.find_element_by_xpath('//*[@id="EIOverviewContainer"]/div/div[1]/div[1]/span').text
                except:
                    description = "N/A"

            #Gather Mission - handling "Read More" button    
                try:
                    read_more = driver.find_element_by_class_name('css-1tgo67c.e16x8fv00') #button class
                    read_more.click()
                    time.sleep(2)
                    mission = driver.find_element_by_xpath('//*[@id="EIOverviewContainer"]/div/div[1]/div[2]').text
                except:
                    mission = "N/A"

##-------------------------------- Gathering Variables - Ratings Pop-up --------------------------------##    
            #Webpage layout 1
                try: 
                    driver.find_element_by_xpath('//*[@id="EIOverviewContainer"]/div/div[3]/div[1]/div[2]').click()
                    time.sleep(5) #let page load

                    rating_overall = driver.find_element_by_xpath('//*[@id="reviewDetailsModal"]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div[3]').text
                    rating_DI = driver.find_element_by_xpath('//*[@id="reviewDetailsModal"]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[3]/div/div[3]').text
                    rating_CV = driver.find_element_by_xpath('//*[@id="reviewDetailsModal"]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[2]/div/div[3]').text
                    rating_WL = driver.find_element_by_xpath('//*[@id="reviewDetailsModal"]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[4]/div/div[3]').text
                    rating_SM = driver.find_element_by_xpath('//*[@id="reviewDetailsModal"]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[5]/div/div[3]').text
                    rating_CB = driver.find_element_by_xpath('//*[@id="reviewDetailsModal"]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[6]/div/div[3]').text
                    rating_CO = driver.find_element_by_xpath('//*[@id="reviewDetailsModal"]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[7]/div/div[3]').text

                    time.sleep(np.random.choice([x/10 for x in range(7,22)])) #some time to rest 
            #Webpage layout 2
                except: 
                    driver.get(url) #recalling url
                    driver.find_element_by_xpath('//*[@id="EIOverviewContainer"]/div/div[4]/div[1]/div[2]').click()
                    time.sleep(5) #let page load
                    
                    rating_overall = driver.find_element_by_xpath('//*[@id="reviewDetailsModal"]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[1]/div/div[3]').text
                    rating_DI = driver.find_element_by_xpath('//*[@id="reviewDetailsModal"]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[3]/div/div[3]').text
                    rating_CV = driver.find_element_by_xpath('//*[@id="reviewDetailsModal"]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[2]/div/div[3]').text
                    rating_WL = driver.find_element_by_xpath('//*[@id="reviewDetailsModal"]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[4]/div/div[3]').text
                    rating_SM = driver.find_element_by_xpath('//*[@id="reviewDetailsModal"]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[5]/div/div[3]').text
                    rating_CB = driver.find_element_by_xpath('//*[@id="reviewDetailsModal"]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[6]/div/div[3]').text
                    rating_CO = driver.find_element_by_xpath('//*[@id="reviewDetailsModal"]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[7]/div/div[3]').text

                    time.sleep(np.random.choice([x/10 for x in range(7,22)])) #some time to rest 
                                        
##---------------------------------------- Creating a Dictionary ----------------------------------------##
                companies.append({ ##UPDATE## this line to create unique dictionary for this scrape attempt
                    "NAME" : name,
                    "SIZE" : size,
                    "LOCATION_HQ" : headquarters,
                    "INDUSTRY" : industry,
                    "RATING_OVERALL" : rating_overall,
                    "RATING_DI" : rating_DI,
                    "RATING_CV" : rating_CV,
                    "RATING_WL" : rating_WL,
                    "RATING_SM" : rating_SM,
                    "RATING_CB" : rating_CB,
                    "RATING_CO" : rating_CO,
                    "NUM_REVIEWS" : num_reviews,
                    "DESCRIPTION" : description,
                    "MISSION" : mission
                                 })

            except: #fail safe for inevitable errors
                unsuccessful_links.append(url) #adding unsuccessful urls to a list ##UPDATE## unsuccessful_links list
                print('ERROR: ', url)
                time.sleep(10)
                
        print(f'Finished scraping {len(companies)} companies') ##UPDATE## companies list name
        df = pd.DataFrame(companies)                           ##UPDATE## df number and companies list name
    return df                                                  ##UPDATE## df number

In [9]:
scraping_pages(100) 



AttributeError: 'WebDriver' object has no attribute 'find_elements_by_tag_name'